# LangChain Cookbook

*This cookbook is based off the [LangChain Conceptual Documentation](https://docs.langchain.com/docs/)*

**Goal:** Provide an introductory understanding of the components and use cases of LangChain via [ELI5](https://www.dictionary.com/e/slang/eli5/#:~:text=ELI5%20is%20short%20for%20%E2%80%9CExplain,a%20complicated%20question%20or%20problem.) examples and code snippets. For use case check another part.

**Links:**
* [LC Conceptual Documentation](https://docs.langchain.com/docs/)
* [LC Python Documentation](https://python.langchain.com/en/latest/)
* [LC Javascript/Typescript Documentation](https://js.langchain.com/docs/)
* [www.langchain.com](https://langchain.com/)
* [LC Twitter](https://twitter.com/LangChainAI)


### **What is LangChain?**
> LangChain is a framework for developing application powered by language models.

**~~TL~~DR**: LangChain makes the complicated parts of working & building with AI models easier. It helps do this in two ways:

1. **Integration** - Bring external data, such as your files, other applications, and api data, to your LLMs.
2. **Agency** - Allow your LLMs to interact with it's environment via decision-making. Use LLMs to help decide which action to take next.

### **Why LangChain?**
1. **Components** - LangChain makes it easy to swap out abstractions and components necessary to work with language models.
2. **Customized Chains** - LangChain provides out of the box support for using and customizing 'chains' - a series of action strung together.
3. **Speed** - This team ships insanely fast. You'll be up to date with latest LLM features.
4. **Community** - Wonderful support.



Through LLMs can be straightforward(text-in, text-out) you'll quickly run into friction points that LangChain helps with once you develop more complicated applications.

*Note: This cookbook will not cover all aspects of LangChain. It's contents have been curated to get you to building & impact as quick as possible. [LangChain Conceptual Documentation](https://docs.langchain.com/docs/)*


In [ ]:
openai_api_key = 'YourAPIKey'

# LangChain Components

## Schema - Nuts and Bolts of Working with LLMs

### **Text**
The natural language way to interact with LLMs

In [ ]:
my_text = "What day comes after Friday"

### **Chat Messages**
Like text, but specified with a message type (System, Human, AI)

* **System** - Helpful background context that tell the AI what to do
* **Human** - Messages that are intended to represent the user
* **AI** - Messages that show what the AI responded with


For more, see OpenAI's [documentation](https://platform.openai.com/docs/guides/chat/introduction)

In [ ]:
!pip install langchain

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=.7, openai_api_key=openai_api_key)

In [ ]:
chat(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out what to eat in one short sentence"),
        HumanMessage(content="I like tomatoes, what should I eat?")
    ]
)

You can also pass more chat history w/ responses from AI

In [ ]:
chat([
    SystemMessage(content="You are a nice AI bot that helps a user figure out what to eat in one short sentence"),
    HumanMessage(content="I like the beaches where should I go?"),
    AIMessage(content="You should go to Nice, France"),
    HumanMessage(content="What else should I do when I'm there?")
])

# Documents

An object that holds a piece to text and metadata (more information about that text)

In [ ]:
from langchain.schema import Document

In [ ]:
Document(page_content="This is my document. It is full of text that I've gathered from other places",
         metadata={
             'my_document_id': 234234,
             'my_document_source': "The LangChain Papers",
             'my_document_create_time': 1680013019
         })

# Models - The Interface to the AI brains

### Language Model

A model that does text in -> text out!

*Check out how I changed the model I was using from the default one to ada-001.
See more models [here](https://platform.openai.com/docs/models)*

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-ada-001", openai_api_key=openai_api_key)

In [ ]:
llm("What day comes after Friday")

### Chat Model
A model that takes a series of messages and returns a message output

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=1, openai_api_key=openai_api_key)

In [ ]:
chat(
    [
        SystemMessage(content="You are an unhelpful AI bot that makes a joke at whatever the user says"),
        HumanMessage(content="I would like to go to New York, how should I do this?")
    ]
)

# Text Embedding Model
Change your text into a vector (a series of numbers that hold the semantic `meaning` of your text). Mainly used when comparing two pieces of text together.

*BTW: Semantic means 'relating to meaning in language or logic.'*

In [ ]:
from langchain.embeddings import OpenAIEmbeddings


embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [ ]:
text = "Hi! it's time for the beach"

In [ ]:
text_embedding = embeddings.embed_query(text)
print(f"Your embedding is length {len(text_embedding)}")
print(f"Here's a sample: {text_embedding[:5]}...")

## Prompts - Text generally used as instruction to your model

### **Prompt**
What you'll pass to the underlying model

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

prompt = """
Today is Monday, tomorrow is Wednesday.

What is wrong with that statement
"""

llm(prompt)

### **Prompt Template**
An object that helps create prompts based on a combination of user input, other non-static information and a fixed template string.

Thing as it as an [f-string](https://realpython.com/python-f-strings/) in python but for prompts.

In [ ]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

template = """
I really want to travel to {location}, What should I do there?

Respond in one short sentence
"""

prompt = PromptTemplate(
    input_variables = ["location"],
    template = template
)

final_prompt = prompt.format(location='Rome')

print(f"Final prompt: {final_prompt}")
print("-----------")
print(f"LLM Output: {llm(final_prompt)}")

### **Example Selectors**
An easy way to select from a series of examples that allow you to dynamic place in-context information into your prompt. Often used when your task is nuanced or you have a large list of examples.

CHeck out different types of example selectors [here](https://python.langchain.com/en/latest/modules/prompts/example_selectors.html)


In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input: {input}\nExample Output: {output}",
)

# Examples of locations that nouns are found
examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "tree", "output": "ground"},
    {"input": "bird", "output": "nest"},
]

In [ ]:
# SemanticSimilarityExampleSelector will select examples that are similar to your input by semantic meaning

example_selector = SemanticSimilarityExampleSelector.from_examples(

    # This is the list of examples available to select from.
    examples,

    # This is the embedding class used to produce embedding which are used to measure semantic similarity.
    OpenAIEmbeddings(openai_api_key=openai_api_key),

    # This is the VectorStore class that is used to store the embedding and do a similarity search over.
    FAISS,

    # This is the number of examples to produce
    k=2
)

In [8]:
similar_prompt = FewShotPromptTemplate(
    # The object that will help select examples
    example_selector=example_selector,

    # Your Prompt
    example_prompt=example_prompt,

    # Customizations that will be added to the top and bottom of your prompt
    prefix="Give the location an item is usually found in",
    suffix="Input: {noun}\nOutput:",

    # What inputs your prompt will receive
    input_variables=["noun"],
)

NameError: name 'FewShotPromptTemplate' is not defined

In [ ]:
# Select a noun!
my_noun = "student"
print(similar_prompt.format(noun=my_noun))

In [ ]:
llm(similar_prompt.format(noun=my_noun))